##Import Libraries

In [ ]:
import pandas as pd
import numpy as np

##İmport Dataset and get dependent / independent values


In [ ]:
dataset = pd.read_csv('insurance.csv')
y = dataset.iloc[:,-1].values # ilk sutun son sutun
X = dataset.iloc[:,:-1].values # ilk sutun son sutun

##Encoding Sex and Smoker columns with using LabelEncoder


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
 
X[:,1] = le.fit_transform(X[:,1]) #male = 0, female = 1
X[:,4] = le.fit_transform(X[:,4]) #no = 1, yes = 0
X[:,5] = le.fit_transform(X[:,5]) #region

##Split the dataset into the train set and test set (2nd Operation)

In [ ]:
def splitData(X):
  global randomstate

  # Split the dataset
  from sklearn.model_selection import train_test_split
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state = randomstate)
  
  # Feature Scaling
  from sklearn.preprocessing import StandardScaler
  sc = StandardScaler()
  X_train[:,:] = sc.fit_transform(X_train[:,:])
  X_test[:,:] = sc.transform(X_test[:,:])
  
  return decisionTreeRegression(X_train,y_train,X_test,y_test)

##Random Forest Regression (3rd Operation)

In [ ]:
def randomForestRegression(X_train,y_train,X_test,y_test):
  from sklearn.ensemble import RandomForestRegressor
  global randomstate
  regressor = RandomForestRegressor(n_estimators = 10,random_state=randomstate)
  regressor.fit(X_train, y_train)
  y_pred = regressor.predict(X_test)
  return calculateScores(y_test,y_pred)

##Decision Tree Regression (3rd Operation)

In [ ]:
def decisionTreeRegression(X_train,y_train,X_test,y_test):
  from sklearn.tree import DecisionTreeRegressor
  global randomstate
  regressor = DecisionTreeRegressor(random_state = randomstate)
  regressor.fit(X_train, y_train)
  y_pred = regressor.predict(X_test)
  return calculateScores(y_test,y_pred)

##KNN Regression (3rd Operation)

In [ ]:
def knnRegression(X_train,y_train,X_test,y_test):
  from sklearn.neighbors import KNeighborsRegressor
  regressor = KNeighborsRegressor()
  regressor.fit(X_train,y_train)
  y_pred = regressor.predict(X_test)
  return calculateScores(y_test,y_pred)

##Calculate R2 Score, MAE and MSE(4th Operation)

In [ ]:
def calculateScores(y_test,y_pred):
  
  
  from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
  
  #r2
  r2Score = r2_score(y_test, y_pred) 
  
  #mae
  mae = mean_absolute_error(y_test, y_pred) 

  #mse
  mse = mean_squared_error(y_test,y_pred,squared = False)

  return [r2Score,round(mae,4),round(mse,4)]

##Send Features (1st Operation)


In [ ]:
def getFeatures():
  global indexes,X,maxR2,r2_features,minMAE,mae_features,minMSE,mse_features

  # print selected values
  print("Features: ",dataset.columns.values[indexes])

  # get features values
  values = X[:,indexes] 

  # send features values and get R2, MAE and MSE scores to the scores list. (indexes: 0 = r2, 1 = mae, 2 = mse)
  scores = splitData(values) 
  
  # print scores
  print("R2  score: ",round(scores[0],5),"\nMAE score: ",scores[1],"\nMSE score: ",scores[2],"\n")

  
  # compare current r2 score and max r2 score
  if(scores[0]>maxR2): 
    maxR2 = scores[0]
    r2_features.clear() # remmove the current features
    r2_features.append(dataset.columns.values[indexes]) # add current selected features
  
  # if current r2 score = max r2 score 
  elif(scores[0] == maxR2):
    r2_features.append(dataset.columns.values[indexes]) #add new features to the matrix


  # compare current mae score and min mae score
  if(scores[1]<minMAE):
    minMAE = scores[1]
    mae_features.clear()
    mae_features.append(dataset.columns.values[indexes])
  # if current mae score == min mae score
  elif(scores[1] == minMAE):
    mae_features.append(dataset.columns.values[indexes])

  # compare current mse score and min mse score
  if(scores[2]<minMSE):
    minMSE = scores[2]
    mse_features.clear()
    mse_features.append(dataset.columns.values[indexes])

  # if current mse score == min mse score  
  elif(scores[2] == minMSE):
    mse_features.append(dataset.columns.values[indexes])

##Select Features

In [ ]:
# always split and train the same random values for each loop, otherwise the algorithm will learn with different values by distinguishing differently in each loop.
from random import randint
randomstate = randint(0,2**32-1) # some goods: 1,245,748,328 || 2,064,061,974 || 3,546,591,408

# define maximum R2 score
maxR2 = 0
# it takes the features where the r2 score is highest.
r2_features = []

import sys
# define min MAE Score and MSE Score
minMAE = sys.maxsize
mae_features = [] # it takes the features where the mae score is lowest.

minMSE = sys.maxsize
mse_features = [] # it takes the features where the mae score is lowest.

# select features  (63 rows : 6 15 20 15 6 1)
for i in range(0,6):
  indexes = [i]
  getFeatures()
  for j in range(-6+i,-1):
    indexes = [i,j]
    getFeatures()
    for k in range(j+1,-1):
      indexes = [i,j,k]
      getFeatures()
      for m in range(k+1,-1):
        indexes = [i,j,k,m]
        getFeatures()
        for n in range(m+1,-1):
          indexes = [i,j,k,m,n]
          getFeatures()
          for s in range(n+1,-1):
            indexes = [i,j,k,m,n,s]
            getFeatures()


# Print Results
print("\n")        
print("Result of ?".center(50,"*"))


print(f"\nMax R2 Score: {round(maxR2,5)}")
print("R2 Features:")
for i in range(0,len(r2_features)):
  print(r2_features[i])

print("".center(25,"-"))


print(f"\nMin MAE Score: {minMAE}")
print("MAE Features:")
for i in range(0,len(mae_features)):
  print(mae_features[i])

print("".center(25,"-"))


print(f"\nMin MSE Score: {minMSE}") # minMSE:,   = virgüllü yazdırma
print("MSE Features:")
for i in range(0,len(mse_features)):
  print(mse_features[i])

print("".center(25,"-"))


print(f"\nrandom state:  {randomstate:,}")